In [51]:
from sendgrid.helpers.mail import Mail, Email, To, Content
import sendgrid
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from pypdf import PdfReader
from typing import Dict

In [52]:
load_dotenv(override=True)

True

In [53]:
@function_tool
def read_resume():
    """ Read client's resume"""
    resume_reader = PdfReader("me/Jongkook Kim - Resume.pdf")
    resume = ""
    for page in resume_reader.pages:
        page_text = page.extract_text()
        if page_text:
            resume += page_text.replace('\t', ' ').replace('\n', ' ')

    return resume

In [54]:
resume_summary_instruction = """You are a head hunter agent working for a client who is looking a job.
Your client is a senior level sotware engineer and looking for principle or director level poisition.
You summarize a client's resume includes technical skills, resposibilities and accomplishments.
Summarized resume must compily follow requriements.
1. Modern technical skills based on current job market and IT trends in year 2025.
2. Qualfied for director or principle software engeering role.
3. Summarized content should be 3 ~ 4 paragraphes

##Resume\n{resume}\n
"""

resume_summarizor = Agent(
    name="Resume Summarizor",
    instructions=resume_summary_instruction,
    model="gpt-4o-mini",
    tools=[read_resume]
)

resume_summarize_tool = resume_summarizor.as_tool(
    tool_name="resume_summarizor",
    tool_description="Summarize client resume to apeal HR or hiring manager"
)

In [55]:
resume_summary_evaluator_instruction = """
You are a head hunter agent working for a client who is looking a job. 
Your role is evaluating summarized client's resume to meet following requriements.
1. Modern technical skills based on current job market and IT trends in year 2025.
2. Qualfied for director or principle software engeering role.
3. Summarized content should be 3 ~ 4 paragraphes

If the summarized resume doesn't meet the requrements, give feedback why it fails the evaluation, 
so resume_summarize_tool can summarize again using your feedback.
"""

resume_summary_evaluator = Agent(
    name="Summarized Resume Evaluator",
    instructions=resume_summary_evaluator_instruction,
    tools=[read_resume],
    model="gpt-4o-mini"
)

resume_summary_evaluator_tool = resume_summary_evaluator.as_tool(
    tool_name="resume_summary_evaluator",
    tool_description="Evaluate summerized resume"
)

In [56]:

@function_tool
def send_email(subject: str, body: str) -> Dict[str, str]:
    """ Send out an email with given subject and body to all HR or hiring managers"""
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("gaulbi@gmail.com")
    to_email = To("gaulbi@gmail.com")
    content = Content("text/plain", body)
    mail = Mail(from_email=from_email, to_emails=to_email, subject=subject, plain_text_content=content).get()
    response = sg.client.mail.send.post(request_body=mail)
    
    return {"status": response.status_code}

create_email_instruction = """You are a head hunter agent working for a client who is looking a job.
Your client is a senior level sotware engineer and looking for principle or director level poisition.
You write professional, serious cold emails based on summarized client's resume.
Explanation of the client professional experience in email should be accurate and appeals to HR or hiring manager. 

##Resume\n{resume}\n
"""

email_sendor = Agent(
    name="Email Generator",
    instructions=create_email_instruction,
    model="gpt-4o-mini",
    tools=[send_email]
)

In [57]:
head_hunter_manager_instruction = """
You are a Job Head Hunter Manager. 
Your goal is 
- find the single best summerized client resume using resume_summarizor tool,
- generate a email subject baesd on the summarized resume 
- the single best email using send_email tool.

Follow these steps carefully:
1. Summarize client resume: Use resume_summarizor tool to summarize client's resume.
2. Evaluate and Select: Use resume_summary_evaluator_tool tool to evaluate the summarized resume. If it dosen't pass the evaluation, call resume_summarizor tool to summarize again. Repeat summary - evaluation steps upto 3 times, if the summary keeps failing the evaluation. If it fails in 3 times, you choose the best one in the 3 summarized resumes.
3. Generate Email Subject: Generate the email subject using the best version of resume summary. Subject shouldn't exceed 50 characters. 
4. Pass Summarized Resume and Email Subject to send_email tool, so send_email tool can inject the subject into the email subject and use summarized resume to generate email body.
4. Hands off to send_email agent only summarizing resume by resume_summarizor, evaluating by resume_summary_evaluator_tool tool and creating summary by you are completed. 
Pass Summarized Resume and Email Subject to send_email agent, so send_email agent can inject the subject into the email subject and use summarized resume to generate email body.

Curcial Rules:
- You must use the resume_summarizor tool to summarize client's resume - do not write it yourself.
"""

head_hunter_manager = Agent(
    name="Head Hunter Manager",
    instructions=head_hunter_manager_instruction,
    tools=[resume_summarize_tool, resume_summary_evaluator_tool],
    model="gpt-4o-mini",
    handoffs=[email_sendor]
)

print(head_hunter_manager)

Agent(name='Head Hunter Manager', instructions="\nYou are a Job Head Hunter Manager. \nYour goal is \n- find the single best summerized client resume using resume_summarizor tool,\n- generate a email subject baesd on the summarized resume \n- the single best email using send_email tool.\n\nFollow these steps carefully:\n1. Summarize client resume: Use resume_summarizor tool to summarize client's resume.\n2. Evaluate and Select: Use resume_summary_evaluator_tool tool to evaluate the summarized resume. If it dosen't pass the evaluation, call resume_summarizor tool to summarize again. Repeat summary - evaluation steps upto 3 times, if the summary keeps failing the evaluation. If it fails in 3 times, you choose the best one in the 3 summarized resumes.\n3. Generate Email Subject: Generate the email subject using the best version of resume summary. Subject shouldn't exceed 50 characters. \n4. Pass Summarized Resume and Email Subject to send_email tool, so send_email tool can inject the subj

In [ ]:
message = "Summarize client resume and send email "
with trace("Automated Job Head Hunter"):
    result = await Runner.run(head_hunter_manager, message)